In [17]:
from googleapiclient.discovery import build
import json

# Load the secrets from the JSON file
with open('client_secrets.json', 'r') as file:
    secrets = json.load(file)

# Set up the API key and YouTube API client
api_key = secrets['api_key']  
youtube = build('youtube', 'v3', developerKey=api_key)

In [18]:
import pandas as pd

def get_video_ids(query, max_results=100):
    video_ids = []
    next_page_token = None

    # Add a keyword to the query to search specifically for Shorts
    modified_query = query + " #shorts"  # or just " shorts"

    while len(video_ids) < max_results:
        # Fetch search results
        request = youtube.search().list(
            part="id",
            q=modified_query,
            type="video",
            maxResults=50,  # Adjust as needed (max 50 per request)
            pageToken=next_page_token
        )
        response = request.execute()

        # Extract video IDs
        for item in response.get('items', []):
            video_ids.append(item['id']['videoId'])
            if len(video_ids) >= max_results:
                break

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    # Create a DataFrame
    df = pd.DataFrame({'Query': [query], 'Video_IDs': [video_ids]})
    return df


In [28]:
# Example use
video_ids = get_video_ids("car")  # Replace with your desired search query and optional max results (standard is 100)


In [29]:
print(video_ids)

  Query                                          Video_IDs
0   car  [5PH0jXruu5E, HVhF_ty_ZT0, 6tZn6Jgha3w, mBzbMo...


In [30]:
curr_video_ids = pd.read_csv('video_ids.csv')

In [31]:
df_merged = pd.concat([curr_video_ids, video_ids], ignore_index=True)
df_merged.head()

,Query,Video_IDs
0,funny,"['P_1-Cpo1P0A', 'FRdgTgKxnx4', '7dUY12c4Xwo', ..."
1,car,"[5PH0jXruu5E, HVhF_ty_ZT0, 6tZn6Jgha3w, mBzbMo..."


In [21]:
df_merged.to_csv('video_ids.csv', index=False)  # Save to CSV file